##### Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Install / Import / Load

This code depends on [Lucid](https://github.com/tensorflow/lucid) (our visualization library), and [svelte](https://svelte.technology/) (a web framework). The following cell will install both of them, and dependancies such as TensorFlow. And then import them as appropriate.

In [2]:
#!npm install -g svelte-cli@2.2.0
#!pip install --user scikit-image

import numpy as np
import tensorflow as tf
from skimage.transform import resize
from skimage.io import imsave

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url
from lucid.misc.gradient_override import gradient_override_map
import lucid.scratch.web.svelte as lucid_svelte

from top5gen.examples.inception_pretrained import get_top_five

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


# Attribution Code

In [3]:
# This, obviously, loads up the model
model = models.InceptionV1()
model.load_graphdef()

In [4]:
# Here we take in the 1000-dimension vector of potential image labels
labels_str = read("https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt")
labels = [line[line.find(" "):].strip().encode() for line in labels_str.decode().split("\n")]
labels = [label.decode().strip().split()[1].replace("_", " ") for label in labels]
labels = ["dummy"] + labels

In [5]:
def raw_class_spatial_attr(img, layer, label, override=None):
  """How much did spatial positions at a given layer effect a output class?"""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts = T(layer).eval()
    
    # display results

    
    if label is None: return np.zeros(acts.shape[1:-1])
    
    # Compute gradient
    softmax_result = T("softmax2_pre_activation")
    score = softmax_result[0, labels.index(label)] # The "score" is the softmax result at idx [0, label idx]
    t_grad = tf.gradients([score], [T(layer)])[0]   
    grad = t_grad.eval({T(layer) : acts})
    
    # Linear approximation of effect of spatial position
    return np.sum(acts * grad, -1)[0]

In [6]:
def raw_spatial_spatial_attr(img, layer1, layer2, override=None):
  """Attribution between spatial positions in two different layers."""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts1 = T(layer1).eval()
    acts2 = T(layer2).eval({T(layer1) : acts1})
    
    # Construct gradient tensor
    # Backprop from spatial position (n_x, n_y) in layer2 to layer1.
    n_x, n_y = tf.placeholder("int32", []), tf.placeholder("int32", [])
    layer2_mags = tf.sqrt(tf.reduce_sum(T(layer2)**2, -1))[0]
    score = layer2_mags[n_x, n_y]
    t_grad = tf.gradients([score], [T(layer1)])[0]
    
    # Compute attribution backwards from each positin in layer2
    attrs = []
    for i in range(acts2.shape[1]):
      attrs_ = []
      for j in range(acts2.shape[2]):
        grad = t_grad.eval({n_x : i, n_y : j, T(layer1) : acts1})
        # linear approximation of imapct
        attr = np.sum(acts1 * grad, -1)[0]
        attrs_.append(attr)
      attrs.append(attrs_)
  return np.asarray(attrs)

In [7]:
def orange_blue(a,b,clip=False):
  """This is what makes the orange/blue highlights in the saliency map"""
  
  # Clip *is* called on each run in the given code
  # --This appears to cut down on noise (in purple) in the final output
  if clip:
    a,b = np.maximum(a,0), np.maximum(b,0)
    
  # This is all original to what I was given, I don't know what the constants represent
  arr = np.stack([a, (a + b)/2., b], -1)
  arr /= 1e-2 + np.abs(arr).max()/1.5
  arr += 0.3  # Brightens the image a tad
  i = 0
  j = 0
  
  for values in arr:
    for value in values:
        for a in value:
          if a == 0.3: i += 1
          j += 1
  
  return [arr, i/j]

# Simple Attribution Example

In [14]:
layers = ['mixed3a', 'mixed3b', 'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e', 'mixed5a', 'mixed5b']

# Write the labels out to the file first
with open ("normalized_file_data.csv", "w") as out:
    out.write(",,,'mixed3a','mixed3b','mixed4a','mixed4b','mixed4c','mixed4d','mixed4e','mixed5a','mixed5b'\n")
    
# Get our top five preds
cur_labels = get_top_five()

# Hold the current file name
file_name = ""

for label in cur_labels:
    # Hold the file path name
    file_name = label[0]
    
    # Note: Must save the file off each time because it will not re-read raw ndarray
    img = resize(load(label.pop(0)), (224, 224))  # Make all 224x224
    imsave("current.jpeg", img, plugin='pil', format_str='jpeg')
    img = load("current.jpeg")
    for tag in label:
        # Print confidence and the label associated
        with open("normalized_file_data.csv", 'a') as out:
            out.write(file_name + ",")  # File name
            out.write(tag.split()[2].strip()[:-1] + ",")  # prob
            out.write(tag.split(':')[-1].strip() + ",")  # class
            
        saliency_layers = []
        saliency_values = []
        for layer in layers:       
            # Get our saliency layer
            saliency = orange_blue(
              raw_class_spatial_attr(img, layer, tag.split(':')[-1].strip(), override=None),
              raw_class_spatial_attr(img, layer, None, override=None),
              clip=True
            )
            saliency_layers.append(saliency[0])
            saliency_values.append(saliency[1])

            attrs = raw_spatial_spatial_attr(img, layer, "mixed5b", override=None)
            attrs = attrs / attrs.max()
            
        # Add each saliency value to the file
        with open("normalized_file_data.csv", 'a') as out:
            # Normalize the saliency values list
            saliency_values = list(map(lambda sv: 100 - sv * 100, saliency_values))
            max_cov = max(saliency_values)
            min_cov = min(saliency_values)
            saliency_values = list(map(lambda sv: (sv - min_cov) / (max_cov - min_cov), saliency_values))
            for sv in saliency_values:
                out.write(str(sv) + ",")
            out.write("\n")

Load conv1_7x7_s2 weights!
Load conv1_7x7_s2 biases!
Load conv2_3x3_reduce weights!
Load conv2_3x3_reduce biases!
Load conv2_3x3 weights!
Load conv2_3x3 biases!
Load inception_3a_1x1 weights!
Load inception_3a_1x1 biases!
Load inception_3a_3x3_reduce weights!
Load inception_3a_3x3_reduce biases!
Load inception_3a_3x3 weights!
Load inception_3a_3x3 biases!
Load inception_3a_5x5_reduce weights!
Load inception_3a_5x5_reduce biases!
Load inception_3a_5x5 weights!
Load inception_3a_5x5 biases!
Load inception_3a_pool_proj weights!
Load inception_3a_pool_proj biases!
Load inception_3b_1x1 weights!
Load inception_3b_1x1 biases!
Load inception_3b_3x3_reduce weights!
Load inception_3b_3x3_reduce biases!
Load inception_3b_3x3 weights!
Load inception_3b_3x3 biases!
Load inception_3b_5x5_reduce weights!
Load inception_3b_5x5_reduce biases!
Load inception_3b_5x5 weights!
Load inception_3b_5x5 biases!
Load inception_3b_pool_proj weights!
Load inception_3b_pool_proj biases!
Load inception_4a_1x1 weigh

/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
